![Two data scientists working on a dashboard.](hr-image-small.png)

A common problem when creating models to generate business value from data is that the datasets can be so large that it can take days for the model to generate predictions. Ensuring that your dataset is stored as efficiently as possible is crucial for allowing these models to run on a more reasonable timescale without having to reduce the size of the dataset.

You've been hired by a major online data science training provider called *Training Data Ltd.* to clean up one of their largest customer datasets. This dataset will eventually be used to predict whether their students are looking for a new job or not, information that they will then use to direct them to prospective recruiters.

You've been given access to `customer_train.csv`, which is a subset of their entire customer dataset, so you can create a proof-of-concept of a much more efficient storage solution. The dataset contains anonymized student information, and whether they were looking for a new job or not during training:

| Column                   | Description                                                                      |
|------------------------- |--------------------------------------------------------------------------------- |
| `student_id`             | A unique ID for each student.                                                    |
| `city`                   | A code for the city the student lives in.                                        |
| `city_development_index` | A scaled development index for the city.                                         |
| `gender`                 | The student's gender.                                                            |
| `relevant_experience`    | An indicator of the student's work relevant experience.                          |
| `enrolled_university`    | The type of university course enrolled in (if any).                              |
| `education_level`        | The student's education level.                                                   |
| `major_discipline`       | The educational discipline of the student.                                       |
| `experience`             | The student's total work experience (in years).                                  |
| `company_size`           | The number of employees at the student's current employer.                       |
| `company_type`           | The type of company employing the student.                                       |
| `last_new_job`           | The number of years between the student's current and previous jobs.             |
| `training_hours`         | The number of hours of training completed.                                       |
| `job_change`             | An indicator of whether the student is looking for a new job (`1`) or not (`0`). |

In [51]:
# Import necessary libraries
import pandas as pd

# Load the dataset
ds_jobs = pd.read_csv("customer_train.csv")

# View the dataset
# ds_jobs.head(15)

In [52]:
# Create a copy of ds_jobs for transforming
ds_jobs_transformed = ds_jobs.copy()

# Start coding here. Use as many cells as you like!

In [53]:
len(ds_jobs_transformed)

19158

In [54]:
ds_jobs_transformed.columns

Index(['student_id', 'city', 'city_development_index', 'gender',
       'relevant_experience', 'enrolled_university', 'education_level',
       'major_discipline', 'experience', 'company_size', 'company_type',
       'last_new_job', 'training_hours', 'job_change'],
      dtype='object')

In [55]:
ds_jobs_transformed.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19158 entries, 0 to 19157
Data columns (total 14 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   student_id              19158 non-null  int64  
 1   city                    19158 non-null  object 
 2   city_development_index  19158 non-null  float64
 3   gender                  14650 non-null  object 
 4   relevant_experience     19158 non-null  object 
 5   enrolled_university     18772 non-null  object 
 6   education_level         18698 non-null  object 
 7   major_discipline        16345 non-null  object 
 8   experience              19093 non-null  object 
 9   company_size            13220 non-null  object 
 10  company_type            13018 non-null  object 
 11  last_new_job            18735 non-null  object 
 12  training_hours          19158 non-null  int64  
 13  job_change              19158 non-null  float64
dtypes: float64(2), int64(2), object(10)
me

### Columns containing categories with only two factors must be stored as Booleans (bool).

In [56]:
# Assuming ds_jobs_transformed is already created as a copy from another DataFrame
# Create a dictionary of columns containing ordered categorical data
ordered_cats = {
    'enrolled_university': ['no_enrollment', 'Part time course', 'Full time course'],
    'education_level': ['Primary School', 'High School', 'Graduate', 'Masters', 'Phd'],
    'experience': ['<1'] + list(map(str, range(1, 21))) + ['>20'],
    'company_size': ['<10', '10-49', '50-99', '100-499', '500-999', '1000-4999', '5000-9999', '10000+'],
    'last_new_job': ['never', '1', '2', '3', '4', '>4']
}

# Loop through each column in the DataFrame
for column in ds_jobs_transformed.columns:
    # Check if the column has exactly 2 unique values
    if ds_jobs_transformed[column].nunique() == 2:
        # Convert the column to Boolean
        ds_jobs_transformed[column] = ds_jobs_transformed[column].astype('bool')
    # Check if the column has exactly the int64 format
    elif ds_jobs_transformed[column].dtype == 'int64':
        # Convert the column to int32
        ds_jobs_transformed[column] = ds_jobs_transformed[column].astype('int32')
    # Check if the column's dtype is a float type
    elif pd.api.types.is_float_dtype(ds_jobs_transformed[column]):
        # Convert the column to float16
        ds_jobs_transformed[column] = ds_jobs_transformed[column].astype('float16')
    # This checks if the current column exists as a key in the dictionary ordered_cats.    
    elif column in ordered_cats.keys():
        # This creates a CategoricalDtype object
        # ordered_cats[column] accesses the list of categories associated with the column in ordered_cats. For example, if the column is 'experience', it retrieves ['<1', '1', '2', ...].
        # ordered=True specifies that these categories have a defined order. This is useful for ordinal data (e.g., experience levels, education levels) where the order of categories matters.
        # Example: If column = 'experience', then this line will create a categorical type where the categories are ordered as: category = pd.CategoricalDtype(['<1', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11-20', '>20'], ordered=True)
        category = pd.CategoricalDtype(ordered_cats[column], ordered=True)
        # This line converts the specified column in the DataFrame ds_jobs_transformed to the newly defined ordered categorical type.
        ds_jobs_transformed[column] = ds_jobs_transformed[column].astype(category)
    # Convert remaining columns to standard categories
    else:
        ds_jobs_transformed[column] = ds_jobs_transformed[column].astype('category')   

ds_jobs_transformed.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19158 entries, 0 to 19157
Data columns (total 14 columns):
 #   Column                  Non-Null Count  Dtype   
---  ------                  --------------  -----   
 0   student_id              19158 non-null  int32   
 1   city                    19158 non-null  category
 2   city_development_index  19158 non-null  float16 
 3   gender                  14650 non-null  category
 4   relevant_experience     19158 non-null  bool    
 5   enrolled_university     18772 non-null  category
 6   education_level         18698 non-null  category
 7   major_discipline        16345 non-null  category
 8   experience              19093 non-null  category
 9   company_size            13220 non-null  category
 10  company_type            13018 non-null  category
 11  last_new_job            18735 non-null  category
 12  training_hours          19158 non-null  int32   
 13  job_change              19158 non-null  bool    
dtypes: bool(2), category(9

In [57]:
len(ds_jobs_transformed)

19158

In [58]:
# Filter students with 10 or more years experience at companies with at least 1000 employees
ds_jobs_transformed = ds_jobs_transformed[(ds_jobs_transformed['experience'] >= '10') & (ds_jobs_transformed['company_size'] >= '1000-4999')]